In [1]:
## Packages
'''Import all the necessary packages and modules'''


%matplotlib qt4 
#uses matplotlib qt4 interface

import hyperspy.api as hs #call hyperspy by hs
hs.preferences.gui(toolkit="traitsui") #opens traitsui preference window
from scipy import signal
import scipy as sp
import numpy as np # package for playing with matrices
import matplotlib.pyplot as plt
import pandas as pd # package for math
import sys, os # python modules for interacting with computer
from tkinter import filedialog # interface for opening a file explorer
from tkinter import * # * imports all files in tkinter 
from tqdm import tqdm
from skimage.morphology import extrema

In [2]:
## Data Directory GUI
'''Getting data directory so data can easily be imported 
using this format data = hs.load('*.dm3', stack=True) to import all dm3s in folder as a stack'''


root = Tk() # Tk() is a function in tkinter that opens a window
root.directory = filedialog.askdirectory() # opens explorer window so you can find the folder of choice
root.withdraw() # closes the tkinter window since it's unnecessary
oldcwd = os.getcwd() # saves old called working directory (place where data is drawn from) as oldcwd use os.chdir(oldcwd) to go back
os.chdir(root.directory) # sets new directory
newcwd = os.getcwd() # saves new directory name as newcwd

In [3]:
## Load Data
''' Open data and set up for analysis'''


data = hs.load('*.dm3', stack=True) # loads all dm3 in the folder directory into a stack called data
s = data.align2D()  # Aligns data stack and removes edges. Not working well
n = np.shape(data)[2] # gives dimension of data (n x m) n = x-axis
m = np.shape(data)[1] # gives dimension of data (n x m) m = y-axis
z = np.shape(data)[0] # gives dimension of time steps = t-axis

In [4]:
##  Definitions of functions needed

# Gets potential outliers indexes based on percentiles
def get_outlier(value, L, U):
    """Check if value is an outlier based on percentiles
    """
    p25 = np.percentile(value, L)
    p75 = np.percentile(value, U)    
    
    lower = p25 - 1.5 * (p75 - p25)
    upper = p75 + 1.5 * (p75 - p25)
    loutlier, uoutlier= value <= lower , value>= upper
    return loutlier, uoutlier # Give 2 output variables - e.g. lower , upper =is_outlier(data,5,95)

# Gets the local mean around a specific index in the matrix
# with size 1=3x3 -center point = 8 points; 2=5x5 -pt = 24 folds over edge ! didn't work over 2
# Now works with any size 
def local_mean(matrix, x_axis, y_axis, size):
    local_data=[]
    numb=[]
    nsize=matrix.shape[0]-1
    msize=matrix.shape[1]-1
    for n in range(-size, size+1,1) :
        for m in range(-size, size+1,1):
            if m==0 and n==0:
                continue
                
            if  x_axis+n>=nsize and   y_axis+m>=msize:  #Cycles around far corner back to 0,0 for arbitrary size
                k = (x_axis+n) 
                k = k % nsize
                l = (y_axis+m) 
                l = l % msize
                local_data.append(matrix[k, l])
                numb.append([k, l])
                continue
            
            if x_axis+n>=nsize:
                k = x_axis+n 
                k = k% nsize
                local_data.append(matrix[k, y_axis + m])
                numb.append([k,y_axis + m])
                continue
            if   y_axis+m>=msize:
                l = (y_axis+m) 
                l = l % msize
                local_data.append(matrix[x_axis +n, l])
                numb.append([x_axis, l])
                continue
          
            local_data.append(matrix[x_axis+n,y_axis+m])
            numb.append([x_axis+n,y_axis+m])
            
        
    loc_mean=np.mean(local_data)        
        
    return loc_mean
  
    

In [5]:
data.plot() # use h to set brightness and contrast

In [ ]:
## Analysis for data cube
'''
Replaces all data outside percentiles get_outlier(data1,1,99) with a local mean.
The local mean size is set by N=number of pixels in each direction N=1 -> (3x3)-1=8 pixels; N=2 -> (5x5)-1=24 pixels 
Effectively autothresholds while removing some x-rays.
Repeat to remove further outliers (e.g 1 >outliers>1000 but local spot [ 5 10 999 13 4])
'''
for t in tqdm(range(0,z)):
    #data=np.random.rand(3,4) used for testing purposes
    repeat=1
    r=0
    while r <repeat:
        data1 = data.data[t,:,:]
        lower, upper = get_outlier(data1,5,95) # sets up 2 truth tables for the positions below lower bound or above upper bound
        c, d = np.where(lower), np.where(upper) # gets indexes of where upper and lower are true
        a, b = np.transpose(c), np.transpose(d) # rather than array [ x values] , array [y- values] gives array [x,y]
        N=3

        for [i,j] in a: # looks at positions where true only - faster than looking at all values      
            loc_mean=local_mean(data1, i, j, N)
            data1[i,j] = loc_mean # replaces all values where true with the local mean

        for i,j in b:
            loc_mean=local_mean(data1, i, j, N)       
            data1[i,j] = loc_mean # replaces all values where true with the local mean

        data.data[t,:,:]=data1
        
        r+=1 

data=hs.signals.Signal2D(data)  #Converts data back to a Signal2D so hyperspy will work for it



In [18]:
## Analysis for data cube
'''
Uses skimage.morphology extrema to find local maximum and minimum replace these points with a local mean.
The local mean size is set by N=number of pixels in each direction N=1 -> (3x3)-1=8 pixels; N=2 -> (5x5)-1=24 pixels 
Effectively autothresholds while removing some x-rays.
Repeat to remove further outliers (e.g 1 >outliers>1000 but local spot [ 5 10 999 13 4])
'''
data2=data
for t in tqdm(range(0,z)):
#     data=np.random.rand(20,20) #used for testing purposes
#     data[3,5] = 50

    data1 = data.data[t,:,:]

    lower = extrema.local_minima(data1)
    upper = extrema.local_maxima(data1)
    c, d = np.where(lower), np.where(upper) # gets indexes of where upper and lower are true
    a, b = np.transpose(c), np.transpose(d) # rather than array [ x values] , array [y- values] gives array [x,y]
    N=3

    for [i,j] in a: # looks at positions where true only - faster than looking at all values      
        loc_mean=local_mean(data1, i, j, N)
        data1[i,j] = loc_mean # replaces all values where true with the local mean

    for i,j in b:
        loc_mean=local_mean(data1, i, j, N)       
        data1[i,j] = loc_mean # replaces all values where true with the local mean

    data.data[t,:,:]=data1
#     data = data1
data=hs.signals.Signal2D(data)  #Converts data back to a Signal2D so hyperspy will work for it

In [19]:
data

array([[  28.5388031 ,  107.10797119,   78.38976288, ...,  460.05697632,
         478.19714355,  472.41000366],
       [ 186.39149475,   67.51266479,   66.26174164, ...,  453.95230103,
         449.25961304,  509.3734436 ],
       [  32.72006989,   50.81036758,   16.25526428, ...,  424.92721558,
         454.94519043,  459.56948853],
       ..., 
       [  83.91295624,   39.55205154,   22.04410744, ...,   28.30703354,
         116.26087189,   91.66529083],
       [  22.02653885,   95.53860474,   54.22337723, ...,   33.70259857,
         139.32774353,  113.13525391],
       [ 146.93943787,   23.9737606 ,   71.58887482, ...,  200.0324707 ,
         174.65901184,  158.34983826]], dtype=float32)

In [9]:
asdf.plot()

C:\Users\kbs1\Documents\HyperSpy WinPython Bundle v1.3.0\python-3.5.1.amd64\lib\site-packages\matplotlib\axes\_base.py:2917: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0.0, right=0.0
  'left=%s, right=%s') % (left, right))


In [ ]:
asdf=data2-data 
asdf=
print(asdf)

In [ ]:
print(a)

In [ ]:
print(data.align2D())

In [ ]:
len(s)

In [ ]:
show_progressbar